In [1]:
%%bash
set -e

cd /content

# Download micromamba into /content/bin/micromamba
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# Create a clean env (isolated from Colab’s global packages)
./bin/micromamba create -y -n cb311 -c conda-forge python=3.11 pip

echo "✅ micromamba ready at /content/bin/micromamba"
echo "✅ env created: cb311"


bin/micromamba


Transaction

  Prefix: /root/.local/share/mamba/envs/cb311

  Updating specs:

   - python=3.11
   - pip


  Package               Version  Build                 Channel           Size
───────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────

  + _openmp_mutex           4.5  20_gnu                conda-forge     Cached
  + bzip2                 1.0.8  hda65f42_9            conda-forge     Cached
  + ca-certificates   2026.2.25  hbd8a1cb_0            conda-forge     Cached
  + icu                    78.2  h33c6efd_0            conda-forge     Cached
  + ld_impl_linux-64     2.45.1  default_hbd61a6d_101  conda-forge     Cached
  + libexpat              2.7.4  hecca717_0            conda-forge     Cached
  + libffi                3.5.2  h3435931_0            conda-forge     Cached
  + libgcc               15.2.0  he0feb66_18           conda-forge     Cached
  

In [2]:
%%bash
set -euo pipefail

cd /content
MICROMAMBA="/content/bin/micromamba"

ts() { date +"[%Y-%m-%d %H:%M:%S]"; }

echo "$(ts) Sanity check micromamba:"
ls -lah "$MICROMAMBA"

echo "$(ts) Upgrading pip tooling inside cb311..."
"$MICROMAMBA" run -n cb311 python -m pip install -U pip setuptools wheel --progress-bar on

echo "$(ts) Installing PyTorch 2.5.1 (CUDA 12.1)... (this can take a while)"
"$MICROMAMBA" run -n cb311 pip install \
  --progress-bar on \
  torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvision==0.20.1+cu121 \
  --index-url https://download.pytorch.org/whl/cu121

echo "$(ts) Installing ONNX (wheel)..."
"$MICROMAMBA" run -n cb311 pip install --progress-bar on onnx==1.16.0

echo "$(ts) Installing Chatterbox package (from GitHub, no-cache, upgrade)..."
"$MICROMAMBA" run -n cb311 pip uninstall -y chatterbox-tts chatterbox || true
"$MICROMAMBA" run -n cb311 pip install \
  --no-cache-dir --upgrade \
  --progress-bar on \
  "chatterbox-tts @ git+https://github.com/devnen/chatterbox-v2.git@master"

echo "$(ts) ✅ Installation complete!"


[2026-02-26 22:03:01] Sanity check micromamba:
-rwxrwxr-x 1 root root 17M Feb 11 14:12 /content/bin/micromamba
[2026-02-26 22:03:01] Upgrading pip tooling inside cb311...
[2026-02-26 22:03:02] Installing PyTorch 2.5.1 (CUDA 12.1)... (this can take a while)
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp311-cp311-linux_x86_64.whl (780.5 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp311-cp311-linux_x86_64.whl (3.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp311-cp311-linux_x86_64.whl (7.3 MB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)

  Running command git clone --filter=blob:none --quiet https://github.com/devnen/chatterbox-v2.git /tmp/pip-install-y_pebwuj/chatterbox-tts_d5fe3a1ba5044b778e404ede64035fed
  Running command git checkout -b master --track origin/master
  Switched to a new branch 'master'
  Branch 'master' set up to track remote branch 'master' from 'origin'.


In [3]:
%%bash
set -e

/content/bin/micromamba run -n cb311 python - <<'PY'
import inspect, torch
import chatterbox.tts_turbo as t

print("✅ torch:", torch.__version__)
print("✅ cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ gpu:", torch.cuda.get_device_name(0))

print("✅ chatterbox.tts_turbo path:", t.__file__)

src = inspect.getsource(t.ChatterboxTurboTTS.from_pretrained)
print("\n--- from_pretrained() (first ~80 lines) ---")
print("\n".join(src.splitlines()[:80]))

# Heuristic check for the common buggy pattern that forces token=True semantics
markers = [" or True", "token=True", "token = True", "use_auth_token=True"]
hits = [m for m in markers if m in src]
print("\nHeuristic auth-forcing markers found:", hits)

if hits:
    raise SystemExit(
        "\n❌ This install still appears to force HF auth.\n"
        "Re-run Cell 2 (it already uses --no-cache-dir --upgrade).\n"
    )

print("\n✅ Looks good: Turbo should download without requiring user tokens.")
PY

✅ torch: 2.5.1+cu121
✅ cuda available: True
✅ gpu: Tesla T4
✅ chatterbox.tts_turbo path: /root/.local/share/mamba/envs/cb311/lib/python3.11/site-packages/chatterbox/tts_turbo.py

--- from_pretrained() (first ~80 lines) ---
    @classmethod
    def from_pretrained(cls, device) -> 'ChatterboxTurboTTS':
        # Check if MPS is available on macOS
        if device == "mps" and not torch.backends.mps.is_available():
            if not torch.backends.mps.is_built():
                print("MPS not available because the current PyTorch install was not built with MPS enabled.")
            else:
                print("MPS not available because the current MacOS version is not 12.3+ and/or you do not have an MPS-enabled device on this machine.")
            device = "cpu"

        local_path = snapshot_download(
            repo_id=REPO_ID,
            token=os.getenv("HF_TOKEN") or False,
            # Optional: Filter to download only what you need
            allow_patterns=["*.safetensors"

In [6]:
%%bash

echo "Step 1 — Clone Perth repo"
cd /content
rm -rf perth
git clone https://github.com/resemble-ai/perth.git

echo "Step 2 — Install from local source"
/content/bin/micromamba run -n cb311 pip install ./perth

echo "Step 3 — Test import"
/content/bin/micromamba run -n cb311 python -c "import perth; print('OK:', dir(perth))"

Step 1 — Clone Perth repo
Step 2 — Install from local source
Processing ./perth
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached matplotlib-3.10.8-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (52 kB)
  Using cached praat_parselmouth-0.4.7-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.9 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tensorboard-2.20.0-p

Cloning into 'perth'...


In [ ]:
# @title 4. Install Server + Run With Full Live Logs (Patched Stable Version)
import os, subprocess, socket, requests, pathlib
from pathlib import Path

PORT = 8004
REPO_DIR = "/content/Chatterbox-TTS-Server"
LOG_STDOUT = "/content/chatterbox_server_stdout.log"

def sh(cmd, check=False):
    return subprocess.run(["bash", "-lc", cmd], check=check)

def port_open(host="127.0.0.1", port=PORT, timeout=0.25):
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except OSError:
        return False

os.chdir("/content")

# Fresh clone
sh("rm -rf /content/Chatterbox-TTS-Server", check=False)
sh("git clone https://github.com/devnen/Chatterbox-TTS-Server.git", check=True)
os.chdir(REPO_DIR)

print("=== Quick system checks ===")
sh("nvidia-smi || true", check=False)

print("\n=== Installing server requirements ===")
if Path("requirements-nvidia.txt").exists():
    sh("/content/bin/micromamba run -n cb311 pip install -U pip setuptools wheel", check=False)
    sh("/content/bin/micromamba run -n cb311 pip install -r requirements-nvidia.txt", check=False)
else:
    sh(
        "/content/bin/micromamba run -n cb311 pip install -U pip setuptools wheel && "
        "/content/bin/micromamba run -n cb311 pip install "
        "fastapi 'uvicorn[standard]' pyyaml soundfile librosa safetensors "
        "python-multipart requests jinja2 watchdog aiofiles unidecode inflect tqdm "
        "pydub audiotsm praat-parselmouth peft",
        check=False
    )

# ---------------------------------------------------
# 🔧 PATCH: Disable Perth watermarker (fix Colab crash)
# ---------------------------------------------------
print("\n=== Patching chatterbox turbo (disable Perth watermarker) ===")
t3_file = pathlib.Path("/root/.local/share/mamba/envs/cb311/lib/python3.11/site-packages/chatterbox/tts_turbo.py")

if t3_file.exists():
    text = t3_file.read_text()
    if "PerthImplicitWatermarker" in text or "apply_watermark" in text:
        text = text.replace(
            "self.watermarker = perth.PerthImplicitWatermarker()",
            "self.watermarker = None  # Patched: disable Perth watermarker for Colab"
        )
        text = text.replace(
            "self.watermarker.apply_watermark(wav, sample_rate=self.sr)",
            "wav"
        )
        t3_file.write_text(text)
        print("✓ Patched Perth watermarker in micromamba env")
    else:
        print("✓ Patch not needed")
else:
    print("⚠ Could not locate tts_turbo.py in micromamba env")


# ---------------------------------------------------

print("\n=== Removing old stdout log ===")
Path(LOG_STDOUT).unlink(missing_ok=True)

print("\n=== Starting server with LIVE logs ===")
print("Log file:", LOG_STDOUT)
print("To stop the server, interrupt this cell.\n")

env = os.environ.copy()
env["PYTHONUNBUFFERED"] = "1"

# Modern HF cache only
env["HF_HOME"] = "/content/hf_home"
env.pop("TRANSFORMERS_CACHE", None)
env.pop("HF_HUB_CACHE", None)
Path(env["HF_HOME"]).mkdir(parents=True, exist_ok=True)

proc = subprocess.Popen(
    ["/content/bin/micromamba", "run", "-n", "cb311", "python", "-u", "server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env=env,
)

with open(LOG_STDOUT, "w", encoding="utf-8", errors="replace") as f:
    shown_link = False
    while True:
        line = proc.stdout.readline()
        if line:
            print(line, end="")
            f.write(line)
            f.flush()

        if (not shown_link) and port_open():
            shown_link = True
            print("\n=== Server port is reachable ===")
            from google.colab.output import serve_kernel_port_as_window
            serve_kernel_port_as_window(PORT)

            # Verify model load status
            try:
                mi = requests.get(f"http://127.0.0.1:{PORT}/api/model-info", timeout=3).json()
                print("\n/api/model-info:", mi)
            except Exception as e:
                print("\n/api/model-info query failed:", repr(e))

        if proc.poll() is not None:
            print("\n=== Server process exited with code", proc.returncode, "===")
            break

=== Quick system checks ===

=== Installing server requirements ===

=== Patching chatterbox turbo (disable Perth watermarker) ===
✓ Patched Perth watermarker in micromamba env

=== Removing old stdout log ===

=== Starting server with LIVE logs ===
Log file: /content/chatterbox_server_stdout.log
To stop the server, interrupt this cell.

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.

0it [00:00, ?it/s]
0it [00:00, ?it/s]
2026-02-26 22:12:19 [INFO] __main__: Starting TTS Server directly on http://0.0.0.0:8004
2026-02-26 22:12:19 [INFO] __main__: API documentation will be available at http://0.0.0.0:8004/docs
2026-02-26 22:12:19 [INFO] __main__: Web UI will be available at http://0.0.0.0:8004/
INFO:     Started server process [10531]
INFO:     Waiting for application startup.
2026-02-26 22:12:19 [INFO] se

<IPython.core.display.Javascript object>


/api/model-info: {'loaded': True, 'type': 'turbo', 'class_name': 'ChatterboxTurboTTS', 'device': 'cuda', 'sample_rate': 24000, 'supports_paralinguistic_tags': True, 'available_paralinguistic_tags': ['laugh', 'chuckle', 'sigh', 'gasp', 'cough', 'clear throat', 'sniff', 'groan', 'shush'], 'turbo_available_in_package': True, 'multilingual_available_in_package': True, 'supports_multilingual': False, 'supported_languages': {'en': 'English'}}
2026-02-26 22:12:49 [INFO] server: Attempting to open web browser to: http://localhost:8004/
2026-02-26 22:12:54 [INFO] server: Request received for main UI page ('/').
2026-02-26 22:12:55 [INFO] server: Request received for /api/ui/initial-data.
2026-02-26 22:12:55 [INFO] utils: Found 28 predefined voices in /content/Chatterbox-TTS-Server/voices
2026-02-26 22:13:15 [INFO] server: Received /tts request: mode='predefined', format='wav'
2026-02-26 22:13:15 [INFO] server: Using predefined voice: Emily.wav
2026-02-26 22:13:15 [INFO] server: Splitting text 

In [ ]:
%%bash
PORT=8004

echo "PIDs listening on port $PORT:"
sudo lsof -t -i:$PORT || true

echo "Killing..."
sudo lsof -t -i:$PORT | xargs -r sudo kill -9

echo "Verify nothing is listening:"
sudo lsof -i:$PORT || true
